In [1]:
#if (!requireNamespace("BiocManager", quietly = TRUE))
#  install.packages("BiocManager")
#BiocManager::install("iterativeBMA")
#install.packages("PDtoolkit")
# install.packages("remotes")
# remotes::install_github("ayhandis/creditR")

##########
# There doesn't appear to be an R implementation of the Spiegelhalter test.
# The SAS implementation seems to be very limited in its scope: it doesn't let you
# specify the hypothesized success probabilities as far as I can tell.

In [16]:
test_data <- read.csv("pd_test_data.csv")

In [13]:
library(iterativeBMA, quietly = T)

brier.score(test_data$predicted_pd, test_data$default_flag)/nrow(test_data)

[1] 0.09727007

In [17]:
library(PDtoolkit, quietly = T)

test_agg <- read.csv("pd_test_agg.csv")
pp.testing(rating.label = test_agg$ratings,
           pdc = test_agg$PD,
           no = test_agg$N,
           nb = test_agg$D, 
           alpha = 0.05)

rating,no,nb,odr,pdc,alpha,binomial,binomial.res,jeffreys,jeffreys.res,zscore,zscore.res,hosmer.lemeshow,hosmer.lemeshow.res
<chr>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<chr>
A,382,33,0.08638743,0.10,0.05,0.8343581,H0: ODR <= PDC,0.8112067,H0: ODR <= PDC,0.8124203,H0: ODR <= PDC,0.7987413,H0: ODR <= PDC
B,515,81,0.15728155,0.15,0.05,0.3397752,H0: ODR <= PDC,0.3177451,H0: ODR <= PDC,0.3217618,H0: ODR <= PDC,0.7987413,H0: ODR <= PDC
C,103,21,0.20388350,0.20,0.05,0.4999238,H0: ODR <= PDC,0.4510711,H0: ODR <= PDC,0.4607546,H0: ODR <= PDC,0.7987413,H0: ODR <= PDC


In [20]:
# Aggregate the results
df2 <- test_data %>%
  group_by(ratings) %>%
  summarise_at(vars(predicted_pd, default_flag), funs(mean(.), sum(.), n()))

# Calculate p-value per rating class
df2$p_value <- pbeta(df2$predicted_pd_mean, 
                     df2$default_flag_sum + 0.5, 
                     df2$predicted_pd_n - df2$default_flag_sum + 0.5)

df2

ratings,predicted_pd_mean,default_flag_mean,predicted_pd_sum,default_flag_sum,predicted_pd_n,default_flag_n,p_value
<chr>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>,<dbl>
A,0.10342906,0.13546798,41.9922,55,406,406,0.01995857
B,0.09952396,0.08541667,47.7715,41,480,480,0.84955196
C,0.09571932,0.08771930,10.9120,10,114,114,0.59864873


In [24]:
pp.testing(rating.label = df2$ratings,
           pdc = df2$predicted_pd_mean,
           no = df2$predicted_pd_n,
           nb = df2$default_flag_sum, 
           alpha = 0.05)

rating,no,nb,odr,pdc,alpha,binomial,binomial.res,jeffreys,jeffreys.res,zscore,zscore.res,hosmer.lemeshow,hosmer.lemeshow.res
<chr>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<chr>
A,406,55,0.13546798,0.10342906,0.05,0.02389227,H1: ODR > PDC,0.01995857,H1: ODR > PDC,0.01700479,H1: ODR > PDC,0.13025,H0: ODR <= PDC
B,480,41,0.08541667,0.09952396,0.05,0.86744061,H0: ODR <= PDC,0.84955196,H0: ODR <= PDC,0.84906675,H0: ODR <= PDC,0.13025,H0: ODR <= PDC
C,114,10,0.08771930,0.09571932,0.05,0.66055279,H0: ODR <= PDC,0.59864873,H0: ODR <= PDC,0.61421823,H0: ODR <= PDC,0.13025,H0: ODR <= PDC


In [15]:
library(creditR, quietly = T)

df <- master.scale(test_data, "default_flag", "predicted_pd")
h <- Herfindahl.Hirschman.Index(df, "Total.Observations")
h

       Final.PD.Range Total.Observations Total.Distr Good.Count Bad.Count
1  <= 0.0001614740971                 25        2.5%         20         5
2     <= 0.0383381406                 75        7.5%         69         6
3    <= 0.04370753681                 25        2.5%         21         4
4    <= 0.04913624977                 25        2.5%         23         2
5    <= 0.05854416783                 50        5.0%         47         3
6    <= 0.06242014789                 25        2.5%         21         4
7    <= 0.06645733297                 25        2.5%         20         5
8    <= 0.07375163657                 50        5.0%         45         5
9    <= 0.08079731541                 50        5.0%         48         2
10   <= 0.08375035232                 25        2.5%         21         4
11   <= 0.08799747431                 25        2.5%         23         2
12   <= 0.09736922493                 75        7.5%         66         9
13    <= 0.1000496439                 

[1] 0.055